In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [2]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from ta import add_all_ta_features
from ta.utils import dropna
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from quiz.models import Pattern, HistoricalPatternActionPoint
from bourse_refs_api.models import Stock, StockHistory

In [4]:
df = pd.DataFrame(StockHistory.objects.all().values())
df = df.sort_values('date').reset_index(drop=True)
df = dropna(df)

In [5]:
stocks = Stock.objects.all()
stocks_object = {
    stock.id: stock for stock in stocks
}

In [6]:
df = df.sort_values(['stock_id', 'date'])

In [7]:
df = df.rename(columns={
    'max_price': 'high',
    'min_price': 'low',
    'last_deal_price': 'close',
    'first_price': 'open'
})

In [8]:
def add_cdl_patterns(group):
    try:
        return pd.concat([group, group.ta.cdl_pattern()], axis=1)
    except:
        return pd.DataFrame()

df = df.groupby('stock_id').apply(add_cdl_patterns)

In [9]:
df = df.rename(columns={ 'CDL_DOJI_10_0.1': 'CDL_DOJI' })

In [10]:
patterns = Pattern.objects.all()

In [15]:
for pattern in tqdm([patterns[0]]):
    df_pattern_key = 'CDL_' + pattern.pattern_key.upper()
    sub_df = df[df[df_pattern_key] != 0].drop(['stock_id'], axis=1).reset_index()[[
        'stock_id', 'date', df_pattern_key
    ]]
    for key, row in tqdm(sub_df.iterrows()):
        try:
            hpap = HistoricalPatternActionPoint(
                stock=stocks_object[row[0]],
                pattern=pattern,
                date=row[1],
                value=row[2]
            )
            hpap.save()
        except Exception as e:
            print(e)
            pass

/var/folders/dr/l8_gry7975l0f3fl8m62sdbh0000gn/T/ipykernel_7794/1656353874.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for pattern in tqdm([patterns[0]]):


  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/dr/l8_gry7975l0f3fl8m62sdbh0000gn/T/ipykernel_7794/1656353874.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for key, row in tqdm(sub_df.iterrows()):


0it [00:00, ?it/s]

no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_api_stock
no such table: main.bourse_refs_ap

In [14]:
HistoricalPatternActionPoint.objects.all()

<QuerySet []>